In [21]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import collections
import os
import time
import inspect
import numpy as np
import tensorflow as tf

In [22]:
def _read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().decode("utf-8").replace("\n", "<eos>").split()

In [23]:
def _build_vocab(filename):
    data = _read_words(filename)
    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    return word_to_id

In [24]:
def _file_to_word_ids(filename, word_to_id):
    data = _read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

In [43]:
def ptb_raw_data(data_path=None):
    train_path = os.path.join(data_path, "ptb.train.txt")
    word_to_id = _build_vocab(train_path)
    train_data = _file_to_word_ids(train_path, word_to_id)
    vocabulary = len(word_to_id)
    return train_data

In [47]:
def ptb_producer(raw_data, batch_size, num_steps, name=None):
    with tf.name_scope(name, "PTBProducer", [raw_data, batch_size, num_steps]):
        raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)
        data_len = tf.size(raw_data)
        batch_len = data_len // batch_size
        data = tf.reshape(raw_data[0 : batch_size * batch_len],
                          [batch_size, batch_len])
        epoch_size = (batch_len - 1) // num_steps
        assertion = tf.assert_positive(
            epoch_size,
            message="epoch_size == 0, decrease batch_size or num_steps")
        with tf.control_dependencies([assertion]):
            epoch_size = tf.identity(epoch_size, name="epoch_size")
        i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
        print(i)
        x = tf.strided_slice(data, [0, i * num_steps],
                             [batch_size, (i + 1) * num_steps])
        x.set_shape([batch_size, num_steps])
        y = tf.strided_slice(data, [0, i * num_steps + 1],
                             [batch_size, (i + 1) * num_steps + 1])
        y.set_shape([batch_size, num_steps])
        return x, y

In [48]:
train_data = ptb_raw_data("./")
ptb_producer(train_data, 20, 20)

Tensor("PTBProducer_1/fraction_of_32_full_Dequeue:0", shape=(), dtype=int32)


(<tf.Tensor 'PTBProducer_1/StridedSlice:0' shape=(20, 20) dtype=int32>,
 <tf.Tensor 'PTBProducer_1/StridedSlice_1:0' shape=(20, 20) dtype=int32>)